# A simple recurrent neural network language model with Keras
by Melissa Roemmele, 4/28/17, roemmele @ ict.usc.edu

### Keras is a Python deep learning framework built on top of Theano, a library for fast math. It lets you put together models with a minimal amount of code, depending on what type of neural network you want to build. Also makes it easy to run models on a GPU. Easy installation with pip ("pip install Keras")

### Keras' documentation isn't bad, but I learned Theano before using Keras. If you have the time, I highly recommend doing a [tutorial on Theano](http://deeplearning.net/tutorial/index.html) just to get a basic sense of how it works. I felt like I understood neural networks better just from doing the logistic regression and multlilayer perceptron tutorials.

### I am going to show how to use Keras to create a language model using a recurrent neural network (RNN). The  input to the network is a bunch of word sequences and the output is a bunch of probabilities, which can then be used to predict new sequences.

In [11]:
import spacy, numpy, random
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU

### Keras has a few text processing utilities built in, including tokenization, but I just discovered the [spacy](https://spacy.io/) library, which is really cool. It does a whole lot more than tokenization (e.g. POS tagging, parsing, semantic indexing, etc.). It's fast and has a really clean API! 

In [12]:
#load spacy model for tokenizing text
encoder = spacy.load('en')

def tokenize(sent):
    #split sentence into words returned as list
    return [word.lower_ for word in encoder(sent)]

### Here's a bunch of text sequences that are input to the RNN. If you're curious, these are a sample from a new dataset, [ROCStories](http://cs.rochester.edu/nlp/rocstories/). The dataset consists of thousands of five-sentence stories, where the task is to predict the final sentence in each story given two choices. Here we'll use the model to generate the final sentence in each of these stories. There are only 100 stories here, which is tiny, but this is for demo purposes.

In [5]:
#input to model is list of sequences, each segmented into sentences

stories = [[u"Dan's parents were overweight.",
  u'Dan was overweight as well.',
  u'The doctors told his parents it was unhealthy.',
  u'His parents understood and decided to make a change.',
  u'They got themselves and Dan on a diet.'],
 [u'Carrie had just learned how to ride a bike.',
  u"She didn't have a bike of her own.",
  u"Carrie would sneak rides on her sister's bike.",
  u'She got nervous on a hill and crashed into a wall.',
  u'The bike frame bent and Carrie got a deep gash on her leg.'],
 [u'Morgan enjoyed long walks on the beach.',
  u'She and her boyfriend decided to go for a long walk.',
  u'After walking for over a mile, something happened.',
  u'Morgan decided to propose to her boyfriend.',
  u"Her boyfriend was upset he didn't propose to her first."],
 [u'Jane was working at a diner.',
  u'Suddenly, a customer barged up to the counter.',
  u'He began yelling about how long his food was taking.',
  u"Jane didn't know how to react.",
  u'Luckily, her coworker intervened and calmed the man down.'],
 [u'I was talking to my crush today.',
  u'She continued to complain about guys flirting with her.',
  u'I decided to agree with what she says and listened to her patiently.',
  u'After I got home, I got a text from her.',
  u'She asked if we can hang out tomorrow.'],
 [u'Frank had been drinking beer.',
  u'He got a call from his girlfriend, asking where he was.',
  u'Frank suddenly realized he had a date that night.',
  u'Since Frank was already a bit drunk, he could not drive.',
  u'Frank spent the rest of the night drinking more beers.'],
 [u'Dave was in the Bahamas on vacation.',
  u'He decided to go snorkeling on his second day.',
  u'While snorkeling, he saw a cave up ahead.',
  u'He went into the cave, and he was terrified when he found a shark!',
  u'Dave swam away as fast as he could, but the shark caught and ate Dave.'],
 [u'Sunny enjoyed going to the beach.',
  u'As she stepped out of her car, she realized she forgot something.',
  u'It was quite sunny and she forgot her sunglasses.',
  u'Sunny got back into her car and heading towards the mall.',
  u'Sunny found some sunglasses and headed back to the beach.'],
 [u'Sally was happy when her widowed mom found a new man.',
  u"She discovered her siblings didn't feel the same.",
  u"Sally flew to visit her mom and her mom's new husband.",
  u'Although her mom was obviously in love, he was nothing like her dad.',
  u"Sally went home and wondered about her parents' marriage."],
 [u'Dan hit his golf ball and watched it go.',
  u'The ball bounced on the grass and into the sand trap.',
  u'Dan pretended that his ball actually landed on the green.',
  u'His friends were not paying attention so they believed him.',
  u'Dan snuck a ball on the green and made his putt from 10 feet.'],
 [u'Josh had a parrot that talked.',
  u'He brought his parrot to school.',
  u"During show and tell, Josh's parrot said a bad word.",
  u'The teacher told Joshua not to bring his bird again.',
  u'When Josh got home, he was grounded.'],
 [u'Hal was walking his dog one morning.',
  u'A cat ran across their path.',
  u"Hal's dog strained so hard, the leash broke!",
  u'He chased the cat for several minutes.',
  u'Finally Hal lured him back to his side.'],
 [u'Brenda was in love with her boyfriend Maxwell.',
  u'He was a successful artist with a promising future.',
  u'Maxwell told Brenda he needed to talk to her.',
  u"She thought he'd propose but he wanted to break up.",
  u'Brenda walked away and now she is the saddest girl out of everyone.'],
 [u'Yanice opened the fridge and found nothing to eat.',
  u'However, there were leftovers.',
  u'She mixed it up in an attempt to make lunch.',
  u'Since the place needed meat, she also fried and eggs.',
  u'She ended up enjoying the meal.'],
 [u'I saw my friend Joe sitting in lobby today.',
  u'I kept him company, as he is a lonely old man.',
  u"He told me he had just listened to Beethoven's Ninth.",
  u'I talked to him for an hour.',
  u'I left him in the lobby and told him I would see him soon.'],
 [u'Twas the night after the first day of junior high.',
  u'Amy and her friend Beth were on the phone.',
  u'They had a lot to catch up on.',
  u'Amy listened patiently as Beth told her about her day.',
  u'She wanted to go 2nd because she knew hers was the better day.'],
 [u'I knew of a young man who won the lottery.',
  u'He used to ride lawn mowers.',
  u'After he won, he went on to using drugs.',
  u'He blew a lot of money.',
  u'Eventually his winnings were revoked after a dui.'],
 [u'A die hard shopper was waiting in the long line outside.',
  u'It was miserably cold.',
  u'The shopper saw a homeless man shivering in the alleyway.',
  u'He gave up his place in the line and brought a gift back from his car.',
  u'The shopper gave the homeless man a nice warm blanket.'],
 [u'Jeff invited his friends over to play board games on Saturday night.',
  u'They arrived at his house early that evening.',
  u'The six of them sat around a big table.',
  u'They took turns deciding which game to play.',
  u'They spent six hours playing different board games.'],
 [u'Chuck reclined on the back porch as he sipped his morning coffee.',
  u'Today he would finally screen in this back porch.',
  u'He gathered his tools and material supplies.',
  u'He labored all day to finish the job.',
  u'That night he snuggled with his wife on the bug free porch.'],
 [u'Jessica decided she wanted to go to the beach.',
  u'She invited all her friends to go along.',
  u'They had a great time, but covered in a lot of sticky sand.',
  u'They searched for a shower for what felt like ages.',
  u'Finally they found one and decided it was the best trip ever.'],
 [u'Dan wanted a pet for Christmas.',
  u'He told his dad.',
  u"His dad listened, but didn't say anything.",
  u'So on Christmas morning, And got a wonderful surprise.',
  u"He received a puppy with a shiny bow on it's head!"],
 [u'Kelly and her friends decided to have a hot dog contest.',
  u'The girls competed against each other.',
  u'They had to make the best tasting one.',
  u'When it was over, Kelly won.',
  u'She won a medal.'],
 [u'Dan was watching a Youtube video.',
  u'His mom was in the kitchen, doing dishes.',
  u'Suddenly, Dan ran into the kitchen and started crying.',
  u'He had just seen a video of a man trampled by an elephant.',
  u'His mom made Youtube off limits to Dan.'],
 [u'Jeff wanted to move out of his house.',
  u'He had no money to pay for a new one.',
  u'One day he bought a scratching ticket.',
  u'He won enough money for a down payment.',
  u'Jeff ended up moving to a new house.'],
 [u'I was walking to school.',
  u"Since I wasn't looking at my feet, I stepped on a rock.",
  u'I landed on the ground in pain.',
  u'Thankfully, a stranger rushed to pick me up.',
  u'He took me to the hospital to seek treatment.'],
 [u'Lily drove into town for some errands.',
  u'While she was there, she bought a large iced coffee to go.',
  u"It was delicious and refreshing and she couldn't wait to finish it.",
  u'She put it on her car roof while fishing for her keys.',
  u'She drove home and the coffee fell off and spilled.'],
 [u'Todd was hungry.',
  u'He did not have anything to cook at his house.',
  u'He decided he need to go buy something to cook.',
  u'On the way to the store Todd decided to make hamburgers.',
  u'Todd buys everything he needed and goes home and cooks.'],
 [u'Virgil brought home a bright blue recliner he had found online.',
  u'His wife thought it was hideous and clashed with the decor.',
  u'Virgil bought some fabric and had it reupholstered.',
  u'His wife complained again, saying bright green still clashed.',
  u'Virgil gave up and threw the recliner away.'],
 [u'Jenna was at the community pool with her family.',
  u'She thought she could go out to the deeper end by herself.',
  u'Without telling anyone, she swam out farther, and lost her footing.',
  u'The lifeguard had to help her out of the water.',
  u'Jenna had learned her lesson.'],
 [u'Joan entered the confessional and kneeled.',
  u'She thought she was confessing to the old parish priest.',
  u'Joan confessed she had fantasized about the young visiting priest.',
  u'Joan felt relief as she left the confessional.',
  u'Then she saw the old parish priest pull up in his car.'],
 [u'Homer decided to go watch a movie.',
  u'But when he entered the movie theater, there was no where to sit.',
  u'He found one spot by a bunch of kids.',
  u'And during the movie, they made lots of noise.',
  u'Homer became so annoyed, he decided to sit in the aisle.'],
 [u'Chip loved dip.',
  u'At one party, he put his chip into the dip and double dipped.',
  u'Skip saw Chip double dip.',
  u'Skip had to flip at seeing the double dip.',
  u'Chip then had to punch Skip on the lip.'],
 [u'Anna wanted to invite her crush Peter to the Sadie Hawkins dance.',
  u'But Peter was very cute and popular.',
  u'Anna feared her was far out of her league.',
  u'She summoned her courage and asked him, expecting a rejection.',
  u'But to her joy, Peter happily agreed to be her date!'],
 [u'Gina had been being mean to the new boy in her class.',
  u'Then a bully began picking on Gina.',
  u'She now knew how the boy felt.',
  u'Gina realized she should stop being mean.',
  u'She realized she should also apologize to the new boy.'],
 [u'Jerry was making toast.',
  u'He set it to medium.',
  u'When the toast came out it was completely burnt.',
  u'He tried other settings with no better results.',
  u'Eventually Jerry bought a new toaster.'],
 [u'Last week I accidentally overdrafted my account.',
  u'A restaurant charged me too much by mistake.',
  u'Afterward I made five more purchases.',
  u'I had hundreds of dollars in overdraft fees.',
  u'My bank refused to reverse more than half.'],
 [u'Frank was laughing so hard he started crying.',
  u"His dog fell into the trash and couldn't get out.",
  u'He took a video of it and posted it online.',
  u'After a week Frank had over 10000 views.',
  u'Frank knew this was as famous as he would ever be.'],
 [u'Ty had been deaf all her life, but now she was hoping to hear.',
  u'Her doctor had offered her a new kind of super-powered hearing aid.',
  u'She had it implanted and then waited eagerly for her first sound.',
  u'And she heard hundreds, voices and music and more!',
  u'She loved those sounds so much that she became a musician.'],
 [u'Bob walked into the ship elevator and heard a voice on the speaker.',
  u'He told his wife the voice sounded the same as his audio book.',
  u'They were surprised to hear the voice in the hallway to their cabin.',
  u'The voice could still be in their cabin, so they called the steward.',
  u"The steward found that the voice was coming from Bob's pocket phone."],
 [u'John is sleepy.',
  u'He starts a pot of coffee.',
  u'John puts cream and sugar in his cup and thermos.',
  u'He then adds coffee to both.',
  u'After finishing the cup, he takes the thermos to work with him.'],
 [u'Susan was excited to plan her first egg hunt in the South.',
  u'She found it fun to hide the candy filled eggs in green grass.',
  u'The sun was high in the sky when the egg hunt started.',
  u'Soon, kids were opening eggs and bursting into tears.',
  u'Susan saw that all the candy inside had melted in the sun.'],
 [u'Bob met Ann and they started dating.',
  u'They got along very well.',
  u'But Bob was Lutheran and Ann was Catholic.',
  u"Ann's mom disapproved of Bob's religion.",
  u"Bob wants to marry Ann someday but is worried he can't."],
 [u"Jen's in laws frustrated her to no end.",
  u'They all went out to have lunch.',
  u'At lunch, the in laws made it a point to ask Jen about her weight.',
  u'Jen told them that she had gained a few pounds in the last two weeks.',
  u'They still worried that Jen looked too thin and offered her some cake.'],
 [u'The woodworker was not satisfied with the cuts from a bit.',
  u'He took the bit from the machine and looked at it.',
  u'The bit had been worn away by a lot of use.',
  u'He took it to a sharpener and began to grind it.',
  u'After a while the old bit was as good as new.'],
 [u'I came out of my class and walked to my locker.',
  u'I pressed my foot on a broken tile and fell on the ground.',
  u'At first, no one was willing to help me up.',
  u'However, one of the teachers around the area helped me get up.',
  u"She took me to the school's nurse."],
 [u'Timmy was always obsessed with airplanes.',
  u'His dream was to be a pilot.',
  u'For his 16th birthday his parents surprised him with flying lessons.',
  u'He loved every minute of it.',
  u"He was sure that's what he wanted to do after that."],
 [u'Last night my wife and I went to the spa.',
  u'We both got relaxing massages.',
  u'We had facials.',
  u'We soaked in warm water and stayed in the sauna.',
  u'It was very nice.'],
 [u'Greg decided to join marching band.',
  u'He practiced for weeks to make sure he made it past auditions.',
  u'He even got his own instrument.',
  u'The day of tryouts came and all his work was for nothing.',
  u"It wasn't that he didn't made it, it's that everyone got accepted."],
 [u'Lisa has a beautiful sapphire ring.',
  u'She always takes it off to wash her hands.',
  u'One afternoon, she noticed it was missing from her finger!',
  u'Lisa searched everywhere she had been that day.',
  u'She was elated when she found it on the bathroom floor!'],
 [u'It was usually hot were Kim lived.',
  u'But everyone was surprised when it was cold one day.',
  u'Kim decided to drink coffee and eat oatmeal.',
  u'So she was glad that the weather was cold.',
  u'But the next day, it was too hot to enjoy hot food and drinks.'],
 [u"A bus driver wanted to save gas so he didn't come to a full stop.",
  u'He slowed down just enough for people to hop on the bus.',
  u'A man jumped at the doorway but missed.',
  u"The bus driver felt that if you fell you didn't deserve a ride.",
  u"He didn't stop to help the man back on the bus."],
 [u'Gary was looking through his fridge for snacks.',
  u'While looking at his food, he noticed everything had small bite marks.',
  u'After looking through his kitchen, he determined he had mice.',
  u"Gary called the local exterminator, who went to Gary's house quickly.",
  u'After the exterminator killed all of the rats, Gary felt peace.'],
 [u'Natalie had auditioned for the lead in the school play.',
  u'She won the part and was super excited.',
  u'She rehearsed for weeks and weeks.',
  u'On opening night, she acted her little heart out.',
  u'The play was a huge success!'],
 [u'Mr Egg was presenting a volcanic eruption to the science class.',
  u'He had a diagram of a volcano that looked like it was made of tinfoil.',
  u'He then took out a huge thing of vinegar, and started to pour it in!',
  u'The class had no clue what was going on and looked on in astonishment.',
  u'The volcano then exploded with substance that looked like lava!'],
 [u"Samantha's dad always taught her how to be self-sufficient.",
  u'He even taught her how to change a tire on a car.',
  u"One day Samantha's tire blew while she was driving.",
  u'She was able to properly change her tire.',
  u'Samantha was very grateful to be able to get home safely.'],
 [u'One day, a pig wandered onto my parents farm.',
  u'I always wanted one for a pet, so I did my best to keep it a secret.',
  u'I kept him in the small shed on the edge of the farm.',
  u'I snuck him food and water and played with him everyday.',
  u'One day, he was gone, but I hope he found his way back home.'],
 [u'Jane passed a small park-like zoo she remembered visiting as a child.',
  u'She turned her car into the park, feeling nostalgic.',
  u'Jane went to see the deer, like the ones she once fed by hand.',
  u'She saw these deer were scrawny, mangy and had terrified eyes.',
  u'Jane wished she had never stopped the car.'],
 [u'It had been a long day.',
  u'Mary was ready to sit back and relax.',
  u'She put on a movie and made some popcorn.',
  u'The movie was much better than she anticipated.',
  u'She was glad she took this time to unwind.'],
 [u'Jill was excited to ski for the first time.',
  u'Her dad took her to the bunny slope.',
  u'She caught on very quickly.',
  u'After about an hour she looked sad.',
  u"When her dad asked why she said because she thought she'd see bunnies."],
 [u'Mark likes to play guitar.',
  u'Mark booked a gig at a local coffeeshop.',
  u'Mark played guitar for 2 hours.',
  u'The 50 people who showed up applauded him.',
  u'Mark packed up his equipment and went home.'],
 [u"Billy's car broke down on the highway.",
  u'He looked under the hood and realized his starter was broken.',
  u'The nearest mechanic quoted Billy 300 dollars, which was far too much.',
  u'He instead called a friend who came and fixed the starter for $100.',
  u'Billy drove away happily with a functioning engine.'],
 [u'Frankie had Christmas shopping to do.',
  u'She went to the store.',
  u'Inside, she walked around looking for gifts.',
  u'Soon her cart was full.',
  u'She paid and took her things home.'],
 [u'Rex had given up on any dreams of becoming a father.',
  u"He was never a good looking man and he didn't have any money.",
  u'One day Rex met a nice woman who liked him despite his shortcomings.',
  u'They became married and eventually had a son.',
  u'Rex is very proud that he is now a father.'],
 [u'Laura had just graduated college.',
  u'She was planning on moving on California.',
  u'She packed all her belongings in her car and drove 18 hours.',
  u'When she arrived at her new apartment she unpacked all her things.',
  u'Laura loved the new change of scenery at her new place.'],
 [u'Mia sat at home in her living room watching sports.',
  u'Her favorite soccer team was playing their rival.',
  u'To encourage her team,  she began chanting positive phrases.',
  u'During her chant, her favorite team scored a goal.',
  u'Mia cheered loudly and thought that she helped score that goal.'],
 [u'Shannon was driving in the highway.',
  u'She then sees a car heading right towards her.',
  u'She has no way of escape.',
  u'The car hits her and both cars are wrecked.',
  u'She is alright though'],
 [u"Nate couldn't stop calling Diana.",
  u'When she arrived in school, she looked in all directions.',
  u'When she saw nate walking, she tried to run to the cafeteria.',
  u"He didn't see her for the whole day.",
  u'She was able to get on the school bus to go home without seeing him.'],
 [u'After her divorce, Sandy spent a lot of time alone.',
  u'Her friends asked her to socialize with them, but she demurred.',
  u'Her friends descended upon her with food and movies to watch.',
  u'They had a very fun evening and Sandy realized she had missed them.',
  u'Sandy was soon back to normal, regularly going out and enjoying life.'],
 [u'Bogart lived on a farm.',
  u'He loved bacon.',
  u'He decided to buy a pig.',
  u'Shortly after, he grew fond of the pig.',
  u'Bogart stopped eating bacon.'],
 [u'The preschoolers were going on a field trip.',
  u'Their teachers took them to the fire station.',
  u'They talked to the firefighters.',
  u'They saw the fire truck.',
  u'It was a wonderful field trip.'],
 [u"Gina's crush sat behind her in class.",
  u'He was rude.',
  u'And way more obnoxious than she had realized.',
  u'She began to dread seeing him.',
  u"Gina realized he wasn't her type after all."],
 [u'Olivia was a ballerina.',
  u'Her dream was to dance internationally.',
  u'One day, the opportunity for an audition came up.',
  u'She went for it, and tried out.',
  u'Amazingly, she won the part.'],
 [u'Tim was a salesman.',
  u'He worked at an electronic store.',
  u'One day he had customers who were unsure.',
  u'He convinced them to buy.',
  u'Tim even convinced them to get an extended warranty.'],
 [u'Today was April Fools Day and everyone played pranks on each other.',
  u'Jeff was sneaking towards Dan, who was sitting down.',
  u'When Jeff turned around, And shouted.',
  u'The chair ended up breaking when Jeff called.',
  u"Dan couldn't help but laugh."],
 [u'Betty had a craving for mint ice cream.',
  u"She went to a local ice cream parlor, but they didn't have mint.",
  u"She went to a grocery store, but they didn't have mint either.",
  u'Betty ended up buying some ice, some cream, and some mint.',
  u'Betty went home and made delicious ice cream herself.'],
 [u'Cade is short, and gets picked on at school.',
  u'His mother tells him he will soon grow very tall like his dad.',
  u'Within a few months Cade had grow a whole inch.',
  u'And within one year Cade was the tallest boy in his class.',
  u'Cade was no longer picked on for being short!'],
 [u'Amber drove home from work one night',
  u'It was really bad weather outside',
  u'She went down a dark road that was covered in water',
  u'she could not tell how deep the water was and drove into a flood',
  u'she messed up her car bad, and it had to be towed'],
 [u'Erica wanted to help her mom this Thanksgiving.',
  u'She wanted to make chicken pot pie for her family.',
  u'She bought all the ingredients at the store.',
  u"When she came home she remembered her oven wasn't working.",
  u"She was able to bake her chicken pot pie at her neighbor's house."],
 [u'Ben came home one day and found a huge mess.',
  u'His plants were knocked over and newspaper was everywhere, shredded.',
  u'Ben called for his dog sternly.',
  u'But his dog was hiding and did not come to him.',
  u'It knew it had done something bad!'],
 [u'Sarah was on a bus to her work.',
  u'She had to pee very badly.',
  u"She couldn't hold it much longer.",
  u'She got off the bus early to pee.',
  u'She caught the next bus to work.'],
 [u'James needed a new pick up truck for work.',
  u'He hauled wooden logs for a living.',
  u'He searched all over on craigslist.',
  u'Finally he found a red truck that he liked.',
  u'James was able to buy the car off the seller for a discount.'],
 [u'Susie sells the 31 products for extra money.',
  u"They are totes that are sold to mother's typically.",
  u'They like them to help bring all their stuff with them.',
  u'It makes for easier travel and they are stylish.',
  u'So far she has done a great job selling them.'],
 [u'Mike was making dinner.',
  u'He was making a pasta and started with the sauce.',
  u'He used tomatoes and fresh vegetables.',
  u'And got fresh herbs from his garden.',
  u'But forgot he had no pasta to heat up!'],
 [u'Today on the view there was a lot of fighting.',
  u'When the women were talking about hot topics they disagreed.',
  u'The war topic made them polarized.',
  u'Rosie and Elizabeth went at it fighting about the war.',
  u'The fight was so bad the producers had to go to splitscreen.'],
 [u'Elliott and Tim were on a high school tennis team.',
  u'They had to play each other for the number 1 ranking spot.',
  u'The match needed to be played, it was pouring rain, and Elliott won.',
  u'Tim complained the next day to the coach about the rain being unfair.',
  u'The coach replied "Was it raining on both sides of the court?"'],
 [u'Today I saw a woman with a baby.',
  u'She was helping the baby eat lunch.',
  u'I was thinking about what it would be like to be feeding a baby.',
  u'I decided that I wanted a baby someday.',
  u'I decided to find a wife so I could start a family.'],
 [u'Louisa and her family took a trip to Epcot.',
  u'The family was super excited.',
  u"They couldn't contain their excitement.",
  u'The moment they got to the park they took pictures.',
  u'At the end of the day they spent ten hours at the park.'],
 [u"Joanie's mom signed her up for swimming lessons at a lake.",
  u'Each day she rode a bus to the lake to take lessons.',
  u'Joanie learned the final test was swimming from a boat to shore.',
  u'She was petrified and prayed to get out of the test.',
  u'On the last day of lessons, the bus broke down and she was spared.'],
 [u'I tried to start jogging last week.',
  u'I got my running shoes on and went out.',
  u'I was excited and ready to go when it started to rainy.',
  u'I turned around and went home instead.',
  u"It hasn't rained since but I haven't wanted to risk it."],
 [u'John woke up sick today.',
  u'He washed his face in the bathroom.',
  u'John went into the kitchen to make some soup.',
  u'He put a bowl of soup into the microwave.',
  u'John dropped the soup when he grabbed it from the microwave.'],
 [u'Tyrese joined a new gym.',
  u'The membership allows him to work out for a year.',
  u'Tyrese got very distracted during the year.',
  u'He lost his job and his grandfather died.',
  u'He lost motivation to go to the gym.'],
 [u'Ryder needed to go outside.',
  u'His owner opened the door for him.',
  u'Ryder played outside.',
  u'He came back in smelling like a dead animal.',
  u'Ryder had to get a bath.'],
 [u'Jimmy just became a police officer in Chicago.',
  u'He is only two weeks into his job and he is nervous.',
  u'Every time he responds to calls he gets very worried.',
  u'His partner told him that the nerves go away in time.',
  u'That news made Jimmy feel a little better.'],
 [u'Tony needed to get gas on his way home.',
  u'He only had enough money to fill half of his tank.',
  u"When he went to pay for it, he was didn't owe anything.",
  u'Someone else had already paid for a full tank of gas for him.',
  u'Tony felt double blessed after getting gas on his way home.'],
 [u'Taylor started working for a man named Mark.',
  u'She wanted to make a good impression on him.',
  u'She began asking questions about him in hopes of finding an affinity.',
  u'She learned that he was a great guitar player.',
  u'Taylor and Mark ended up forming a band together.'],
 [u'We moved from our condo in 2013.',
  u'We had been in our condo since 1987.',
  u'My wife went to the store to buy moving boxes.',
  u'She bought boxes that were too large.',
  u'We had trouble lifting them on moving day, but we managed.'],
 [u'The girl was scared to go outside.',
  u'Her mom encouraged her to go.',
  u'She ended up going.',
  u'She met other kids.',
  u'They were all nice and played with her.'],
 [u'Trevor and his wife were at home on his day off.',
  u'They decided to spend their day watching a movie together.',
  u'Right as they sat down to start the movie, they heard a doorbell.',
  u'Trevor got the door and saw his parents had come to visit.',
  u'They all sat and watched the movie together on the couch.'],
 [u'Sally decided to get a haircut.',
  u'She went to the stylist and got her cut.',
  u'She was upset when she realized the stylist cut it way too short.',
  u'The stylist did not charge her but that did not fix her hair.',
  u'She waited for months for her hair to grow back out.']]

### Let's first figure out the vocabulary the RNN needs to know. I map each word in the training stories to a numerical index. When working with large datasets it's common to filter all words occurring less than a certain number of times, and replace them with some "UNKNOWN" token. 

In [13]:
def make_lexicon(stories):
    '''Create map of words in input sequences to numerical indices'''
    words = []
    for story in stories:
        for sent in story:
            words.extend(tokenize(sent))
    words = set(words)
    lexicon = {word:index for index, word in enumerate(words)}
    return lexicon

lexicon = make_lexicon(stories)
print lexicon

{u'raining': 0, u'all': 1, u'winnings': 2, u'managed': 17, u'saying': 654, u'dance': 5, u'skip': 6, u'when': 1222, u'results': 9, u'laura': 11, u'mile': 13, u'ice': 1060, u'grateful': 15, u'go': 16, u'chair': 18, u'finish': 1225, u'marching': 21, u'hearing': 1092, u' ': 866, u'apartment': 24, u'but': 1167, u'ty': 25, u'swam': 26, u'elizabeth': 557, u'father': 28, u'young': 29, u'herbs': 1257, u'laughing': 1029, u'to': 32, u'finally': 33, u'bike': 34, u'wants': 841, u'starter': 35, u'marks': 36, u'siblings': 38, u'disapproved': 39, u'enjoying': 40, u'hideous': 41, u'started': 1224, u'customer': 43, u'family': 977, u'woman': 44, u'wished': 45, u'garden': 264, u'sitting': 47, u'very': 48, u'hear': 1170, u'blanket': 49, u'gina': 50, u'sufficient': 1233, u'bunnies': 52, u'every': 53, u'got': 448, u'cook': 56, u'join': 57, u'drinking': 58, u'continued': 59, u'minute': 60, u'elated': 62, u'school': 63, u'yanice': 1067, u'parrot': 65, u'did': 66, u'die': 67, u'marry': 68, u'accidentally': 69, 

### I then use the lexicon to convert the stories from text to numerical indices so they can be processed by the RNN.

In [14]:
def encode_sent(sent):
    '''Encode sequences as numerical indices using lexicon'''
    sent = tokenize(sent)
    sent = [lexicon[word] for word in sent]
    return sent

#example of encoded story
story = stories[0]
print story, "\n"
encoded_story = [encode_sent(sent) for sent in story]
print encoded_story

[u"Dan's parents were overweight.", u'Dan was overweight as well.', u'The doctors told his parents it was unhealthy.', u'His parents understood and decided to make a change.', u'They got themselves and Dan on a diet.'] 

[[378, 1237, 988, 566, 528, 831], [378, 1156, 528, 1204, 944, 831], [953, 545, 121, 698, 988, 886, 1156, 491, 831], [698, 988, 1045, 570, 741, 32, 902, 311, 142, 831], [386, 448, 193, 570, 378, 1093, 311, 1084, 831]]


### Now I build an RNN model with four layers: a layer for converting words to distributed vector representations (embeddings), two recurrent layers (I use the GRU variation, keras also provides LSTM or just vanilla RNN), and a prediction layer that will output a probability for each word in the lexicon using the softmax function (each probability indicates the chance of that word being the next word in the sequence). 

### The "stateful=True" parameter for the GRU indicates that the RNN should "remember" the sequence it previously observed, meaning that it will use the existing hidden state as its initial state when it processes the next sequence. This means the model has a chance of learning long-term dependencies between words in different sentences. 

### Keras lets you train in batches of more than one sequence, by having you specify the batch size when you create the model. Batch training significantly speeds up the training process. If you do batch training your sequences will all have to be the same length, which is not the case with our data. People deal with this by padding the sequences with zeros where there are no more words left. To keep it simple here I instead set the batch size to 1.

### If you use batches, you have to specify the length of the batch (equal to the number of words in the longest sequence in the batch, where all other sequences will have extra zeros). The n_timesteps parameter below specifies this. Here because I am only reading one sequence at a time and each sequence has a different length, I can set this parameter to be None and it will accomodate different numbers of timesteps.

### The Dense layer predicts probabilities at a particular timestep (i.e. word). If the input is a sequence, there should be a probability distribution at each timestep, so the TimeDistributed() class around the Dense layer takes care of this. The output will be a sequence of probability distributions. 

### One huge benefit of Keras is that it has several optimization algorithms already implemented. I use Adam here, there are several other available including SGD, RMSprop, and Adagrad. You can change parameters like learning rate and gradient clipping as well.

In [8]:
rnn = Sequential()

lexicon_size = len(lexicon)
n_embedding_nodes = 300
n_hidden_nodes = 500
batch_size = 1
n_timesteps = None

#word embedding layer
embedding_layer = Embedding(batch_input_shape=(batch_size, n_timesteps),
                            input_dim=lexicon_size,
                            output_dim=n_embedding_nodes, 
                            mask_zero=True)
rnn.add(embedding_layer)

#recurrent layers (GRU)
recurrent_layer1 = GRU(output_dim=n_hidden_nodes,
                       return_sequences=True, 
                       stateful=True)
rnn.add(recurrent_layer1)

recurrent_layer2 = GRU(output_dim=n_hidden_nodes,
                       return_sequences=True, 
                       stateful=True)
rnn.add(recurrent_layer2)

#prediction (softmax) layer
pred_layer = TimeDistributed(Dense(lexicon_size, 
                                   activation="softmax"))
rnn.add(pred_layer)

#select optimizer and compile
rnn.compile(loss="sparse_categorical_crossentropy", 
            optimizer='adam')



### To train, I iterate through the stories and feed each story to the model sentence by sentence. Because the RNN is stateful, it will remember the previous sentence when it reads the next one, until reset_states() is called after the final sentence in the story. I track the cross-entropy loss for each epoch to determine how well it is learning - the loss should go down with each epoch.

In [15]:
def train_epoch(stories):  
    losses = []  #track cross-entropy loss during training
    for story in stories:
        prev_eos = None
        for sent in story:
            #encode_sent into indices
            sent = numpy.array(encode_sent(sent))
            if prev_eos:
                '''encode last token in previous sentence so that first word 
                of this sentence is conditioned on it'''
                sent = numpy.insert(sent, 0, prev_eos)
            sent_x = sent[None, :-1]
            sent_y = sent[None, 1:, None]
            loss = rnn.train_on_batch(x=sent_x, y=sent_y)
            losses.append(loss)
            prev_eos = sent[-1]
        #finished story, now clear hidden layer states to read a new story
        rnn.reset_states()
    loss = numpy.mean(losses)
    return loss


n_epochs = 10
print "Training RNN on", len(stories), "stories for", n_epochs, "epochs..."
for epoch in range(n_epochs):
    loss = train_epoch(stories)
    print "epoch {} loss: {:.3f}".format(epoch + 1, loss)

Training RNN on 100 stories for 10 epochs...
epoch 1 loss: 0.218
epoch 2 loss: 0.184
epoch 3 loss: 0.205
epoch 4 loss: 0.259
epoch 5 loss: 0.336
epoch 6 loss: 0.299
epoch 7 loss: 0.258
epoch 8 loss: 0.225
epoch 9 loss: 0.172
epoch 10 loss: 0.135


### Now that the model is trained, it can be used to predict sentences. I will take some stories, give the model their first four sentences, and have it output the probability of the first word the fifth sentence. Then I select the word with the highest probability and add it to the sequence. Alternatively, you can generate the next word through random sampling from the probability distribution (specify mode='random' instead of mode='max'). In either case, I repeat this process, each time predicting the next word based on the sequence so far. I stop generating words either when maximum number of words is reached or an end-of-sentence marker (e.g. punctuation like ".") is generated.

In [17]:
def predict(init_story, max_words, mode='max'):
    '''generate the endings of stories word by word based on word probabilities predicted by rnn'''
    
    eos_tokens = [lexicon[token] for token in [".", "?", "!"] if token in lexicon]
    
    pred_ending = []
    
    '''read initial sentences of story into model'''
    for sent in init_story:
        sent = encode_sent(sent)
        sent = numpy.array(sent)[None, :]
        p_next_word = rnn.predict_on_batch(sent)[0][-1]
       
    '''now start predicting new words'''
    for idx in range(max_words):
        if mode == 'max':
            #generate word with highest probability of being next in this sequence
            next_word = numpy.argmax(p_next_word)
        elif mode == 'random':
            #sample from probability distribution to get next word
            next_word = numpy.random.choice(a=p_next_word.shape[-1], p=p_next_word)
        pred_ending.append(next_word)
        if next_word in eos_tokens:
            #an end-of-sentence marker (e.g. punctuation) was generated, so stop generating
            break
        p_next_word = rnn.predict_on_batch(numpy.array(next_word)[None, None])[0][-1]
    
    rnn.reset_states()
    return pred_ending

'''create lookup table to get words from their indices'''
lexicon_lookup = [word for index, word in
                  sorted([(index, word) for word, index in lexicon.items()])]

for story in random.sample(stories, 15):
    init_story = story[:-1]
    print "INIT STORY:", " ".join(init_story)
    print "GOLD ENDING:", story[-1]
    pred_ending = predict(init_story, max_words=20, mode='random')
    #decode predicted sentence from numerical indices back into words
    pred_ending = [lexicon_lookup[word] for word in pred_ending]
    print "PREDICTED ENDING:", " ".join(pred_ending)
    print "\n"

INIT STORY: I tried to start jogging last week. I got my running shoes on and went out. I was excited and ready to go when it started to rainy. I turned around and went home instead.
GOLD ENDING: It hasn't rained since but I haven't wanted to risk it.
PREDICTED ENDING: it has n't rained since i i n't wanted to risk it .


INIT STORY: A die hard shopper was waiting in the long line outside. It was miserably cold. The shopper saw a homeless man shivering in the alleyway. He gave up his place in the line and brought a gift back from his car.
GOLD ENDING: The shopper gave the homeless man a nice warm blanket.
PREDICTED ENDING: he loved the longer of the parents suddenly a " over and ready to go to the beach .


INIT STORY: Joan entered the confessional and kneeled. She thought she was confessing to the old parish priest. Joan confessed she had fantasized about the young visiting priest. Joan felt relief as she left the confessional.
GOLD ENDING: Then she saw the old parish priest pull up i

## Issues with Keras

### If you like scikit-learn, Keras has a [scikit-learn wrapper](https://keras.io/scikit-learn-api/). Convenient if you want to set up pipelines with more than one model or do cross-validation on several parameters.

### In my opinion, the worst thing about Keras is debugging. Because Theano is compiled, you can't easily step through the network during training and view the values of intermediate layers in the model. Theano lets you run in test mode before the model is compiled, so you can see the results of each function. But as far as I can tell Keras doesn't provide anything special in the way of debugging.

## Alternatives to Keras

### Lasagne: also uses Theano, but possibly less well-maintained than Keras? http://lasagne.readthedocs.io/en/latest/user/installation.html

### TensorFlow: Google's deep learning library. Has a Python, C++, and Go API. Seems to be the overtaking Keras as the most popular Python deep learning library. https://www.tensorflow.org/

### Chainer: seems fast but I've never developed with it. http://chainer.org/

## Stuff I've Found Helpful for Understanding/Implementing RNNs 
#### (this is biased to learning RNNs specifically for text data)

### Among the [Theano tutorials](http://deeplearning.net/tutorial/) mentioned above, there are two specifically on RNNs for NLP: [semantic parsing](http://deeplearning.net/tutorial/rnnslu.html#rnnslu) and [sentiment analysis](http://deeplearning.net/tutorial/lstm.html#lstm)

### [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (same model as shown here, with raw Python code) 

### TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

### This [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTMs work and why they are better than plain RNNs (this explanation also applies to the GRU used here)

### Another [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) that documents well both the theory of RNNs and their implementation in Python (and if you care to implement the details of the stochastic gradient descent and backprogation through time algorithms, which I do not, this is very helpful)